In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)



In [4]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



In [5]:

rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       270
           1       0.95      0.70      0.81        30

    accuracy                           0.97       300
   macro avg       0.96      0.85      0.89       300
weighted avg       0.97      0.97      0.96       300



In [6]:

xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



In [7]:
from imblearn.combine import SMOTETomek
smt=SMOTETomek(random_state=42)
X_train_res,y_train_res=smt.fit_resample(X_train,y_train)

np.unique(y_train_res,return_counts=True)

(array([0, 1]), array([619, 619]))

In [8]:
"""models=[
    (
        "LogisticRegression",
        LogisticRegression(C=1,solver="liblinear"),
        (X_train,y_train),
        (X_test,y_test)
    ),
    (
        "Random Forest",
        RandomForestClassifier(n_estimators=30,max_depth=3),
        (X_train,y_train),
        (X_test,y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False,eval_metric='logloss'),
        (X_train,y_train),
        (X_test,y_test)  
    ),
    (
        "XGBClassifier with SMOTE",
        XGBClassifier(use_label_encoder=False,eval_metric='logloss'),
        (X_train,y_train),
        (X_test,y_test)
    )
]"""

'models=[\n    (\n        "LogisticRegression",\n        LogisticRegression(C=1,solver="liblinear"),\n        (X_train,y_train),\n        (X_test,y_test)\n    ),\n    (\n        "Random Forest",\n        RandomForestClassifier(n_estimators=30,max_depth=3),\n        (X_train,y_train),\n        (X_test,y_test)\n    ),\n    (\n        "XGBClassifier",\n        XGBClassifier(use_label_encoder=False,eval_metric=\'logloss\'),\n        (X_train,y_train),\n        (X_test,y_test)  \n    ),\n    (\n        "XGBClassifier with SMOTE",\n        XGBClassifier(use_label_encoder=False,eval_metric=\'logloss\'),\n        (X_train,y_train),\n        (X_test,y_test)\n    )\n]'

In [9]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [10]:
reports=[]

for model_name,params,model,train_set,test_set in models:
    X_train=train_set[0]
    y_train=train_set[1]
    X_test=test_set[0]
    y_test=test_set[1]

    model.set_params(**params)
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    report=classification_report(y_test,y_pred,output_dict=True)
    reports.append(report)


In [11]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost



In [12]:
mlflow.set_experiment("Anomaly Detection")
mlflow.set_tracking_uri("http://127.0.0.1:5000/")


for i,element in enumerate(models):
    model_name=element[0]
    params=element[1]
    model=element[2]
    report=reports[i]

    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        }) 

        if "XGB" in model_name:
            model._estimator_type = "classifier"
            mlflow.xgboost.log_model(model, name="model")
        else:
            mlflow.sklearn.log_model(model, name="model")  

2026/01/02 20:40:22 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/02 20:40:22 INFO mlflow.store.db.utils: Updating database tables
2026/01/02 20:40:22 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/02 20:40:22 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/01/02 20:40:23 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/02 20:40:23 INFO alembic.runtime.migration: Will assume non-transactional DDL.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/1/runs/7e3226643ea74cc8b57dfdf8ddb18c98
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/1/runs/c1f66d5591db4f77b6627ba652168677
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


2026/01/02 20:41:04 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\raman\AppData\Local\Temp\tmpxgeqk1ev\model, flavor: xgboost). Fall back to return ['xgboost==3.1.2']. Set logging level to DEBUG to see the full traceback. 


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/1/runs/4db35efe479a4b858e4e6ac4f6d871fd
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


2026/01/02 20:41:15 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\raman\AppData\Local\Temp\tmp99q7lhbk\model, flavor: xgboost). Fall back to return ['xgboost==3.1.2']. Set logging level to DEBUG to see the full traceback. 


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/1/runs/3c5cef557a8e436daaf8506368bf36fc
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [15]:
model_name = "XGB-SMOTE"
#run_id=input("Please type RunID")
run_id="3c5cef557a8e436daaf8506368bf36fc"
model_uri = f"runs:/{run_id}/{model_name}"

result=mlflow.register_model(
    model_uri,model_name)

Registered model 'XGB-SMOTE' already exists. Creating a new version of this model...


MlflowException: Unable to find a logged_model with artifact_path XGB-SMOTE under run 3c5cef557a8e436daaf8506368bf36fc

In [ ]:
x=input()